# Stock Clustering

In this module of the financial web app, I will be performing an unsupervised clustering of data on a number of well-known stocks stored in the financial database created. Stocks are clustered on the basis of historical OHLCV data(daily returns) and Technical Inidicators(RSI, MACD etc.), this allows for grouping of stocks based on their behavioural patterns(how they tend to move with time) in the market.

We’re capturing things like:

📈 Trend behavior: Are they in an uptrend or downtrend?

🔁 Momentum: Do they accelerate upward/downward suddenly?

🔄 Mean-reversion vs. breakout: Do they bounce around a mean (like value stocks), or break out like growth stocks?

📊 Volatility dynamics: Do they swing hard or stay tight in range?

📦 Volume reactions: Does volume spike on price moves, or is it steady?



## Libraries

In [13]:
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)

import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)

# Data Visualisation
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_style('darkgrid')

## Parameters and Variables

Setting some default parameters and variables for our whole notebook

In [14]:
font_size = 20
plt.rcParams['axes.labelsize'] = font_size
plt.rcParams['axes.titlesize'] = font_size + 2
plt.rcParams['xtick.labelsize'] = font_size - 2
plt.rcParams['ytick.labelsize'] = font_size - 2
plt.rcParams['legend.fontsize'] = font_size - 2

## Getting the Data

Here's a brief description of the data used here:

| Field      | Description                                         |
| ---------- | --------------------------------------------------- |
| **O**pen   | Price at the beginning of the trading period        |
| **H**igh   | Highest price during the period                     |
| **L**ow    | Lowest price during the period                      |
| **C**lose  | Price at the end of the period                      |
| **V**olume | Number of shares/contracts traded during the period |





| Indicator    | Type               | Measures                     | Use Case                           |
| ------------ | ------------------ | ---------------------------- | ---------------------------------- |
| **RSI**      | Momentum           | Overbought/Oversold          | Entry/exit signals                 |
| **MACD**     | Momentum           | Trend direction & strength   | Bullish/bearish signals            |
| **SMA**      | Trend              | Average price (equal weight) | Trend detection                    |
| **EMA**      | Trend              | Average price (recent bias)  | Faster trend signals               |
| **ATR**      | Volatility         | Magnitude of price movement  | Risk measurement                   |
| **BB Bands** | Volatility + Trend | Price position & squeeze     | Mean-reversion or breakout signals |










In [1]:
# Importing the libraries to get the data from the database
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

# Loading the DB credentials from .env file
load_dotenv()

# Getting the database details and storing it
def get_db_url():
    user = os.getenv("DB_USER")
    password = os.getenv("DB_PASSWORD")
    host = os.getenv("DB_HOST")
    port = os.getenv("DB_PORT","5432")
    db = os.getenv("DB_NAME")
    return f"postgresql://{user}:{password}@{host}:{port}/{db}"



In [2]:
# Loading the data from the database
def load_data(join: bool = False, chunksize: int = None, columns: list = None, limit:int = None):
    engine = create_engine(get_db_url())

    # Building the SELECT clause
    cols = ", ".join(columns) if columns else "*"

    # Defining the query with optional join
    if join:
        query = f"""
        SELECT {cols}
        FROM stock_data sd
        JOIN technical_indicators ti
        ON sd.ticker = ti.ticker and sd.date = ti.date"""

    else:
        query = f"SELECT {cols} FROM stock_data"

    if limit:
        query += f" LIMIT {limit}"

    if chunksize:
        return pd.read_sql(query, engine, chunksize=chunksize)
    else:
        df = pd.read_sql(query,engine)
        engine.dispose()
        return df


In [3]:
# Specifying the columns to be imported
columns = [
    "sd.ticker","sd.date","sd.open","sd.high","sd.low","sd.close","sd.volume",
    "ti.ticker","ti.date","ti.rsi","ti.macd","ti.sma","ti.ema","ti.atr","ti.bb_upper","ti.bb_middle","ti.bb_lower"

]

# Getting the data in batches
data_chunks = load_data(join=True,columns=columns,chunksize=10000)

In [4]:
# Loading the chunks to view them
for chunk in data_chunks:
    print(chunk.head())

  ticker        date    open    high     low   close    volume ticker  \
0   AAPL  2025-05-20  207.67  208.47  205.03  206.86  40940792   AAPL   
1   AAPL  2025-05-19  207.91  209.48  204.26  208.78  46140500   AAPL   
2   AAPL  2025-05-16  212.36  212.57  209.77  211.26  54737900   AAPL   
3   AAPL  2025-05-15  210.95  212.96  209.54  211.45  45029500   AAPL   
4   AAPL  2025-02-10  229.57  230.59  227.20  227.65  33115600   AAPL   

         date      rsi    macd       sma       ema     atr  bb_upper  \
0  2025-05-20  50.9125  0.6616  207.9322  208.2333  6.3560  218.3768   
1  2025-05-19  51.9873  0.6951  206.6456  208.3341  6.6680  218.3950   
2  2025-05-16  54.7913  0.6309  205.5044  208.2227  6.6425  218.9238   
3  2025-05-15  55.0024  0.2873  204.1300  207.4634  6.9380  218.4958   
4  2025-02-10  41.6908 -2.5951  232.7478  230.9541  6.0456  241.4600   

   bb_middle  bb_lower  
0   206.9875  195.5982  
1   206.5860  194.7770  
2   205.8050  192.6862  
3   205.0910  191.6862  
4  

In [6]:
# Loading all the chunks into a single dataframe
full_df = pd.concat(load_data(join=True,chunksize=10000,columns=columns),ignore_index=True)
full_df.head()

,ticker,date,open,high,low,close,volume,ticker,date,rsi,macd,sma,ema,atr,bb_upper,bb_middle,bb_lower
0,AAPL,2025-05-20,207.67,208.47,205.03,206.86,40940792,AAPL,2025-05-20,50.9125,0.6616,207.9322,208.2333,6.3560,218.3768,206.9875,195.5982
1,AAPL,2025-05-19,207.91,209.48,204.26,208.78,46140500,AAPL,2025-05-19,51.9873,0.6951,206.6456,208.3341,6.6680,218.3950,206.5860,194.7770
2,AAPL,2025-05-16,212.36,212.57,209.77,211.26,54737900,AAPL,2025-05-16,54.7913,0.6309,205.5044,208.2227,6.6425,218.9238,205.8050,192.6862
3,AAPL,2025-05-15,210.95,212.96,209.54,211.45,45029500,AAPL,2025-05-15,55.0024,0.2873,204.1300,207.4634,6.9380,218.4958,205.0910,191.6862
4,AAPL,2025-02-10,229.57,230.59,227.20,227.65,33115600,AAPL,2025-02-10,41.6908,-2.5951,232.7478,230.9541,6.0456,241.4600,230.9775,220.4950


In [7]:
# Getting the number of datapoints
print("The number of rows is {} and the number of columns is {}".format(full_df.shape[0],full_df.shape[1]))


The number of rows is 11940 and the number of columns is 17


## Data Cleaning

In this section:

* Data Cleaning
* Feature Engineering

In [15]:
# Getting info on the dataset, to define the cleaning method

full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11940 entries, 0 to 11939
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ticker     11940 non-null  object 
 1   date       11940 non-null  object 
 2   open       11940 non-null  float64
 3   high       11940 non-null  float64
 4   low        11940 non-null  float64
 5   close      11940 non-null  float64
 6   volume     11940 non-null  int64  
 7   ticker     11940 non-null  object 
 8   date       11940 non-null  object 
 9   rsi        11940 non-null  float64
 10  macd       11940 non-null  float64
 11  sma        11940 non-null  float64
 12  ema        11940 non-null  float64
 13  atr        11940 non-null  float64
 14  bb_upper   11940 non-null  float64
 15  bb_middle  11940 non-null  float64
 16  bb_lower   11940 non-null  float64
dtypes: float64(12), int64(1), object(4)
memory usage: 1.5+ MB


**From the above output, we can conclude that:**

* There are no missing values whatsoever
* There are duplicate columns for ticker and data which will be removed
* date is stored as an object not datetime, this will be converted
* ticker is categorical and will be encoded into numerical forms later

In [16]:
# Dropping the duplicate columns
full_df = full_df.loc[:,~full_df.columns.duplicated()]
full_df.head()

,ticker,date,open,high,low,close,volume,rsi,macd,sma,ema,atr,bb_upper,bb_middle,bb_lower
0,AAPL,2025-05-20,207.67,208.47,205.03,206.86,40940792,50.9125,0.6616,207.9322,208.2333,6.3560,218.3768,206.9875,195.5982
1,AAPL,2025-05-19,207.91,209.48,204.26,208.78,46140500,51.9873,0.6951,206.6456,208.3341,6.6680,218.3950,206.5860,194.7770
2,AAPL,2025-05-16,212.36,212.57,209.77,211.26,54737900,54.7913,0.6309,205.5044,208.2227,6.6425,218.9238,205.8050,192.6862
3,AAPL,2025-05-15,210.95,212.96,209.54,211.45,45029500,55.0024,0.2873,204.1300,207.4634,6.9380,218.4958,205.0910,191.6862
4,AAPL,2025-02-10,229.57,230.59,227.20,227.65,33115600,41.6908,-2.5951,232.7478,230.9541,6.0456,241.4600,230.9775,220.4950


### Feature Engineering

Now, we will engineer new features, that will help us in the model training, as it will help the model to better understand patterns



In [20]:
# First of all, we will convert date to datetime
full_df['date'] = pd.to_datetime(full_df['date'])

# Sorting by ticker and date
full_df = full_df.sort_values(by=['ticker','date'])

#### Daily Return

In [21]:
full_df['daily_return'] = full_df.groupby('ticker')['close'].pct_change()

#Grouping now
grouped = full_df.groupby('ticker')

#### Rolling Volatility

In [22]:
full_df['rolling_volatility'] = grouped['daily_return'].rolling(window=3).std().reset_index(level=0, drop=True)

#### Average Volume

In [23]:
full_df['avg_volume'] = grouped['volume'].rolling(window=3).mean().reset_index(level=0, drop=True)


#### Trend Strength

In [24]:
full_df['trend_strength'] = (full_df['close'] - full_df['sma'])/full_df['sma']

#### Volatility Range

In [25]:
full_df['volatility_range'] = full_df['high'] - full_df['low']

In [26]:
# Dropping rows from rolling operations
full_df = full_df.dropna().reset_index(drop=True)

In [28]:
full_df.head()

,ticker,date,open,high,low,close,volume,rsi,macd,sma,ema,atr,bb_upper,bb_middle,bb_lower,daily_return,rolling_volatility,avg_volume,trend_strength,volatility_range
0,AAPL,2023-01-06,126.01,130.29,124.89,129.62,87754700,42.1189,-4.5841,128.1711,128.8381,4.3780,146.7155,133.7440,120.7725,0.036794,0.023754,8.594367e+07,0.011304,5.40
1,AAPL,2023-01-09,130.47,133.41,129.89,130.15,70790800,43.0379,-4.2292,127.9811,129.1005,4.3360,145.5051,133.1190,120.7329,0.004089,0.024263,7.983607e+07,0.016947,3.52
2,AAPL,2023-01-10,130.26,131.26,128.12,130.73,63896200,44.0842,-3.8566,128.0589,129.4264,4.2505,144.2481,132.5475,120.8469,0.004456,0.018777,7.414723e+07,0.020858,3.14
3,AAPL,2023-01-11,131.25,133.51,130.46,133.49,69458900,48.8949,-3.3006,128.8867,130.2391,4.1648,142.3583,131.9975,121.6367,0.021112,0.009724,6.804863e+07,0.035716,3.05
4,AAPL,2023-01-12,133.88,134.26,131.44,133.41,71379600,48.7639,-2.8338,129.3089,130.8733,4.0687,139.7605,131.3945,123.0285,-0.000599,0.011361,6.824490e+07,0.031716,2.82
